# 로또 당첨정보 크롤링

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.select import Select
import time
import re 
import pandas as pd
import numpy as np

In [ ]:
# 전체 리스트 
win_result_list = []
win_date_list = []
total_reward_list = []
each_reward_list = []
num_of_winner_list = []
win_cat_list = []
total_saleprice_list = []

# 크롬드라이버 옵션 지정 : 창 최대화
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')

# 드라이버 실행
driver = webdriver.Chrome('chromedriver.exe',options = options)

# 로또 당첨번호 확인 홈페이지 접속
driver.get('https://dhlottery.co.kr/gameResult.do?method=byWin')

# 회차목록 가져오기
round_list = driver.find_element_by_id('dwrNoList').get_attribute('innerText').split('\n')

# 각 회차별 당첨번호 가져오기
for r in round_list:
    
    # 드롭다운 회차 메뉴 값 변경
    select_round = Select(driver.find_element_by_id('dwrNoList')) 
    select_round.select_by_value(str(r))
    driver.find_element_by_id('searchBtn').click()
    
    # 대기시간 1초
    time.sleep(1)
    
    # 사이트의 url 정보(html)를 가져와 Parsing
    soup = BeautifulSoup(driver.page_source,'html.parser')
    # 회차별 당첨번호 리스트 생성 : 'span' 태그의 'ball'로 시작한는 클래스 text 추출
    win_result = [num.text for num in soup.find_all('span',re.compile('ball.*'))]
    
    win_date = '-'.join(re.compile('\d+').findall(soup.select('p.desc')[0].text))
    
    #당첨금액 리스트 생성
    rewards = [int(re.sub('[,원]','',x.text)) for  x in soup.select('tr > td.tar')]
    total_rewards = []
    each_rewards = []
    for idx, reward in enumerate(rewards):
        # 등수별 전체 당첨금액
        if idx%2==0:
            total_rewards.append(reward)
        # 등수별 1인당 당첨금액
        else :
            each_rewards.append(reward)
    # 당첨자수 
    num_of_winners = [x/y if x !=0 else 0 for x,y in zip(total_rewards,each_rewards)]
    
    # 총판매액
    total_saleprice = int(re.sub('[,원]','',soup.select_one('li > strong').text))
    
    # 1등 당첨 유형
    cat_list= re.compile('\w+\d+').findall(soup.select('td')[5].text)
    win_cats = list(map(lambda x: re.compile('[가-힣]+').search(x).group(),cat_list))
    values = list(map(lambda x: re.compile('\d+').search(x).group(),cat_list))
    # 모든 유형 없을경우 해당 유형 생성 및 값에 0 입력
    none_cats = [cat for cat in ['자동','수동','반자동'] if cat not in win_cats]
    for none_cat in none_cats:
        win_cats.append(none_cat)
        values.append(0)
    # 자동, 수동, 반자동 순으로 값 정렬
    cat_dict  = {cat : value for cat,value in zip(win_cats,values)}
    sorted_values = [int(cat_dict[cat]) for cat in ['자동','수동','반자동']] 
    
    print('*'*20,r,'회차','*'*20)
    print('당첨번호 :',win_result)
    print('당첨날짜 :',win_date)
    print('총 당첨금액 :', total_rewards)
    print('1명당 당첨금액 :',each_rewards)
    print('당첨자수 :',num_of_winners)
    print('판매금액 :',total_saleprice)
    print('유형별 1등 당첨자수 : ',sorted_values)
    print()
    
    # 전체 리스트에 저장
    win_result_list.append(win_result)
    win_date_list.append(win_date)
    total_reward_list.append(total_rewards)
    each_reward_list.append(each_rewards)
    num_of_winner_list.append(num_of_winners)
    total_saleprice_list.append(total_saleprice)
    win_cat_list.append(sorted_values)
    
    
    

# 크롤링 결과 확인

In [ ]:
# 각 리스트 원소 개수 확인
print(len(round_list))
print(len(win_result_list))
print(len(win_date_list))
print(len(total_reward_list))
print(len(each_reward_list))
print(len(num_of_winner_list))
print(len(total_saleprice_list))
print(len(win_cat_list))

In [ ]:
# 당첨번호 6개 + 보너스
win_result_list

In [ ]:
# 당첨날짜
win_date_list

In [ ]:
# 총 당첨금액
total_reward_list

In [ ]:
# 1인당 당첨금액
each_reward_list

In [ ]:
# 당첨자수
num_of_winner_list

In [ ]:
# 총판매액
total_saleprice_list

In [ ]:
# 유형별 1등 당첨자수(자동, 수동, 반자동)
win_cat_list

In [ ]:
# 최종 데이터프레임 생성
lotto_df = pd.DataFrame({'회차' : round_list,
             '당첨번호1' : [num[0] for num in win_result_list],
              '당첨번호2' : [num[1] for num in win_result_list],
              '당첨번호3' : [num[2] for num in win_result_list],
              '당첨번호4' : [num[3] for num in win_result_list],
              '당첨번호5' : [num[4] for num in win_result_list],
              '당첨번호6' : [num[5] for num in win_result_list],
              '당첨번호_보너스' : [num[6] for num in win_result_list],
             '당첨날짜' : win_date_list,
             '총당첨금액_1등' : [total_reward[0] for total_reward in total_reward_list],
              '총당첨금액_2등' : [total_reward[1] for total_reward in total_reward_list],
              '총당첨금액_3등' : [total_reward[2] for total_reward in total_reward_list],
              '총당첨금액_4등' : [total_reward[3] for total_reward in total_reward_list],
              '총당첨금액_5등' : [total_reward[4] for total_reward in total_reward_list],
             '1인당첨금액_1등' : [each_reward[0] for each_reward in each_reward_list],
              '1인당첨금액_2등' : [each_reward[1] for each_reward in each_reward_list],
              '1인당첨금액_3등' : [each_reward[2] for each_reward in each_reward_list],
              '1인당첨금액_4등' : [each_reward[3] for each_reward in each_reward_list],
              '1인당첨금액_5등' : [each_reward[4] for each_reward in each_reward_list],
             '당첨자수_1등' : [num_of_winner[0] for num_of_winner in num_of_winner_list],
              '당첨자수_2등' : [num_of_winner[1] for num_of_winner in num_of_winner_list],
              '당첨자수_3등' : [num_of_winner[2] for num_of_winner in num_of_winner_list],
              '당첨자수_4등' : [num_of_winner[3] for num_of_winner in num_of_winner_list],
              '당첨자수_5등' : [num_of_winner[4] for num_of_winner in num_of_winner_list],
             '판매금액' : total_saleprice_list,
             '1등당첨유형_자동' : [win_cat[0] for win_cat in win_cat_list],
             '1등당첨유형_수동' : [win_cat[1] for win_cat in win_cat_list],
             '1등당첨유형_반자동' : [win_cat[2] for win_cat in win_cat_list]})

In [ ]:
lotto_df.head()

In [ ]:
# 데이터프레임 저장
lotto_df.to_csv('로또당첨정보.csv',encoding='cp949',index = False)

# 추가 : 1등 당첨 판매점 정보 크롤링

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver = webdriver.Chrome('chromedriver.exe',options = options)
driver.get('https://pyony.com/lotto/rounds/latest/')
time.sleep(1)

store_name_list = []
store_cat_list = []
store_add_list = []

for r in round_list:
    
    # 드롭다운 회차 메뉴 값 변경
    select_round = Select(driver.find_element_by_id('id_round'))
    select_round.select_by_visible_text(str(r))
    driver.find_element_by_id('btn_search').click()
    time.sleep(1)
    
    # url정보(html) Paring
    soup = BeautifulSoup(driver.page_source,'html.parser')
    
    # 당첨 판매점 정보 없는 회차 존재 
    # 1등 당첨자 존재하지만 판매점 정보 존재 X -> 코드 실행 중지(1 - 261회 미존재)
    try:
        # 판매점 정보 가져오기
        store_info_list = [x.text.strip() for x in soup.select('tbody')[1].select('td')]
    except:
        # 1등 당첨자가 없을 때 판매점 정보 존재 X -> 다음 회차로 넘어감
        if soup.select('tbody > tr')[0].select_one('td').text =='0':
            continue
        # 1등 당첨자 존재 할 때 판매점 정보 존재 X -> 코드 실행 중지(이전 회차 모두 존재 X)
        else:
            break
    # 판매점 이름
    store_name = store_info_list[0::3]
    # 판매점 당첨 유형
    store_cat = store_info_list[1::3]
    # 판매점 주소
    store_add = store_info_list[2::3]
    
    print('*'*20,r,'회차','*'*20)
    print('1등 판매점 :',','.join(store_name),end='\n\n')
    print('당첨 유형 :',','.join(store_cat),end='\n\n')
    print('주소 :',','.join(store_add),end='\n\n')
    
    # exist_round_list : 판매점 정보 존채 회차 리스트 
    exist_round_list.append(r)
    store_name_list.append(store_name)
    store_cat_list.append(store_cat)
    store_add_list.append(store_add)

In [ ]:
# 최종 데이터프레임 생성
store_df = pd.DataFrame(columns = ['회차','판매점','당첨유형','주소'])

# 판매점 정보 존재 회차 수 만큼반복
for r in range(len(exist_round_list)):
    # 해당 회차에 해당하는 판매점 수 만큼 회차 원소값 복제
    r_list = [exist_round_list[r]] * len(store_name_list[r])
    # 회차별 데이터프레임 생성
    temp_df = pd.DataFrame({'회차' : r_list,
                          '판매점' : store_name_list[r],
                          '당첨유형' : store_cat_list[r],
                          '주소' : store_add_list[r]})
    #최종 데이터프레임에 회차별 데이터프레임 추가
    store_df = pd.concat([store_df,temp_df],axis=0)
store_df.reset_index(drop=True,inplace = True)

In [ ]:
store_df.head(30)

In [ ]:
store_df.to_csv('판매점정보.csv',encoding='cp949',index = False)